In [10]:
import random
#from operator import attrgetter

kolory=["♠️","♥️","♣️","♦️"]
wartości=["A","K","Q","W","10","9","8","7","6","5","4","3","2"]
#awersy
piki=['🂡', '🂢', '🂣', '🂤', '🂥', '🂦', '🂧', '🂨', '🂩', '🂪', '🂫', '🂭', '🂮']
kiery=['🂱', '🂲', '🂳', '🂴', '🂵', '🂶', '🂷', '🂸', '🂹', '🂺', '🂻', '🂽', '🂾']
karo=['🃁', '🃂', '🃃', '🃄', '🃅', '🃆', '🃇', '🃈', '🃉', '🃊', '🃋', '🃍', '🃎']
trefle=['🃑', '🃒', '🃓', '🃔', '🃕', '🃖', '🃗', '🃘', '🃙', '🃚', '🃛', '🃝', '🃞']
jokery=['🂿', '🃏']

class Karta:
    '''definicja karty do gry''' 
    def __init__(self, wartość = "", kolor = ""):
        self.kolor = kolor
        self.wartość = wartość
    def __str__(self):
        return self.nazwa()
    def nazwa(self):
        return self.wartość + self.kolor
 
class Talia(list):
    '''definicja talii kart do gry'''
    def __init__(self,*args):
        '''utworzenie talii kart'''
        super().__init__(*args)
        for kolor in kolory:
            for wartość in wartości:
                self.append(Karta(wartość,kolor))
    
    def __str__(self):
        return f'{self.w_talii()}'

    def daj(self):
        '''losowanie karty z talii'''
        if len(self) > 0:
            wylosowana = random.choice(self)
            self.remove(wylosowana)
        else:
            wylosowana = None
        return(wylosowana)
    
    def liczba_kart(self):
        '''liczba kart w talii'''
        return len(self)
    
    def w_talii(self):
        nazwy_kart=[]
        for karta in self:
            nazwy_kart.append(karta.nazwa())
        return(nazwy_kart)

class Gracz:
    '''definicja grającego'''
    def __init__(self):
        self.ręka = []
    
    def __str__(self):
        return f'{self.w_ręce()}'
    
    def weź(self, karta):
        self.ręka.append(karta)
        
    def w_ręce(self):
        nazwy_kart=[]
        for karta in self.ręka:
            nazwy_kart.append(karta.nazwa())
        return(nazwy_kart)
    
    def get_sorted(self):
        posortowane=sorted(
            self.ręka,
            key=lambda karta:(kolory.index(karta.kolor), wartości.index(karta.wartość)),
            reverse=False
        )
        nazwy_kart=[]
        for karta in posortowane:
            nazwy_kart.append(karta.nazwa())
        return(nazwy_kart)



In [ ]:
talia = Talia()
#print(f'talia pełna: {talia}')
gracz_1 = Gracz()
gracz_2 = Gracz()
gracz_3 = Gracz()
gracz_4 = Gracz()
for _ in range(13):
    gracz_1.weź(talia.daj())
    gracz_2.weź(talia.daj())
    gracz_3.weź(talia.daj())
    gracz_4.weź(talia.daj())

print(f'gracz 2 rozdane:  {gracz_2}')
print(f'gracz_2 sortowane:  {gracz_2.get_sorted()}')

#print(f'gracz 3:  {gracz_3}')
#print(f'gracz 4:  {gracz_4}')
#print(f'talia po rozdaniu: {talia}, pozostało: {talia.liczba_kart()}')


gracz 2 rozdane:  ['K♣️', '2♦️', '4♥️', '4♣️', '6♦️', '8♥️', 'K♥️', '9♠️', 'Q♦️', '3♥️', '8♦️', '3♣️', '2♠️']


AttributeError: 'Gracz' object has no attribute 'w_ręce_posortowane'